In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import censusdata

from scipy import stats

os.environ["CENSUS_API_KEY"] = "d6a829e3229193acd4e98d0f4ad90540cdfe59d5"

# Extract SF Census Data from API

In [17]:
zip_codes = [94401,94402,94403,94497,94404,94063,94062,94061,94064,94065,94117,94121,94122,94188,94404,94123,94403,94164,94146,
94497,94108,94112,94134,94158,94127,94126,94103,94159,94134,94107,94102,94110,94118,94132,94128,94112,94104,94116,
94133,94143,94401,94124,94130,94147,94109,94129,94105,94114,94115,94402,94119,94125,94111,94131,94141,94140, 
             94044, 94002, 94903, 94939, 94015, 94901, 94960, 94066, 94904, 94010, 
              94070, 94947, 94949, 94945, 94080, 94920, 94941, 94965, 94014, 94025, 94925]
zip_codes_str = [str(i) for i in zip_codes]



In [129]:
def local_census_data(year, zip_code):
    # Specify the state code in the geographic boundary
    zipcode = censusdata.censusgeo([('state', '06'), ('zip code tabulation area', zip_code)])

    # Download the data for the specified variables and geographic boundary
    sfbg_zip = censusdata.download('acs5', year, zipcode,
                                   ['B01003_001E','B08013_001E','B25001_001E','B25002_002E','B25002_003E'],
                                   key='d6a829e3229193acd4e98d0f4ad90540cdfe59d5')
    sfbg_zip = sfbg_zip.reset_index()
    sfbg_zip = sfbg_zip.rename(columns={
        'index': 'Zip Code',
        'B01003_001E': 'Total Population',
        'B08013_001E': 'Aggregate Travel Time to Work',
        'B25001_001E': 'Total Housing',
        'B25002_002E': 'Owner-occupied housing units',
        'B25002_003E': 'Renter-occupied housing units',
    })
    sfbg_zip['Percentage of Rent'] = sfbg_zip['Renter-occupied housing units']/sfbg_zip['Total Housing']
    sfbg_zip['Zip Code'] = sfbg_zip['Zip Code'].astype(str)
    sfbg_zip['Zip Code'] = sfbg_zip['Zip Code'].apply(lambda x: str(x).split(">")[-1].split(":")[1].strip() 
                                                                if isinstance(x, str) else x.tract.split(":")[-1].strip())
    sfbg_zip['Year'] = year
    sfbg_zip = sfbg_zip.sort_values(by = ['Zip Code'], ignore_index = True)
    return sfbg_zip


# Concat SF 10 year census data

In [21]:
# Concatenate
sf_all = pd.DataFrame()
start_year = 2011
end_year = 2020

for year in range(2011, 2020):
    for zip_c in zip_codes_str:
        try:
            sf_county = local_census_data(year, zip_c)
            sf_all = pd.concat([sf_all, sf_county], ignore_index=True)
        except Exception:
            pass

In [34]:
sf_all.to_csv('sf_new_all.csv',index=False)

In [33]:
sf_all['Average Travel Time to Work per person'] = sf_all['Aggregate Travel Time to Work']/ 
                                                         sf_all['Total Population']

KeyError: 'Aggregate Travel Time to Work'

In [32]:
sf_all = sf_all.drop('Aggregate Travel Time to Work',axis=1)
sf_all = sf_all.dropna()

,Zip Code,Total Population,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Percentage of Rent,Year,Average Travel Time to Work per person
0,94401,34845,13606,12693,913,0.067103,2011,11.918353
1,94402,23408,9617,9356,261,0.027139,2011,11.343131
2,94403,39102,16190,15266,924,0.057072,2011,11.720244
3,94404,32986,13882,13157,725,0.052226,2011,12.094222
4,94063,30788,9340,8877,463,0.049572,2011,9.511823
...,...,...,...,...,...,...,...,...
562,94941,32009,14139,13290,849,0.060047,2019,14.139461
563,94965,11394,6364,5950,414,0.065053,2019,15.313323
564,94014,49170,14478,14044,434,0.029977,2019,17.490746
565,94025,43392,16365,15106,1259,0.076932,2019,11.890325


In [100]:
duplicates = sf_all.duplicated(subset=['Zip Code', 'Year'])
print(f"Number of duplicate rows: {duplicates.sum()}")
sf_all_drop = sf_all.drop_duplicates(subset=['Zip Code', 'Year'])


Number of duplicate rows: 54


In [79]:
zipcode = list(sf_all_drop['Zip Code'].unique())

In [80]:
for zip_code in zipcode:
    rows_for_zip_code = sf_all_drop[sf_all_drop['Zip Code'] == zip_code]
    print(f"Number of rows for Zip Code {zip_code}: {len(rows_for_zip_code)}")

Number of rows for Zip Code 94401: 9
Number of rows for Zip Code 94402: 9
Number of rows for Zip Code 94403: 9
Number of rows for Zip Code 94404: 9
Number of rows for Zip Code 94063: 9
Number of rows for Zip Code 94062: 9
Number of rows for Zip Code 94061: 9
Number of rows for Zip Code 94065: 9
Number of rows for Zip Code 94117: 9
Number of rows for Zip Code 94121: 9
Number of rows for Zip Code 94122: 9
Number of rows for Zip Code 94123: 9
Number of rows for Zip Code 94108: 9
Number of rows for Zip Code 94112: 9
Number of rows for Zip Code 94134: 9
Number of rows for Zip Code 94158: 9
Number of rows for Zip Code 94127: 9
Number of rows for Zip Code 94103: 9
Number of rows for Zip Code 94107: 9
Number of rows for Zip Code 94102: 9
Number of rows for Zip Code 94110: 9
Number of rows for Zip Code 94118: 9
Number of rows for Zip Code 94132: 9
Number of rows for Zip Code 94116: 9
Number of rows for Zip Code 94133: 9
Number of rows for Zip Code 94124: 9
Number of rows for Zip Code 94130: 9
N

In [53]:
duplicates = sf_all_drop.duplicated(subset=['Zip Code', 'Year'])
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


In [101]:
sf_all_drop.loc[:, 'Average Travel Time to Work per person'] = sf_all_drop.loc[:, 'Average Travel Time to Work per person'].astype(float)


/var/folders/kz/dhtdp71n1q98lc7x715t7c0h0000gp/T/ipykernel_30522/2077931995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_all_drop.loc[:, 'Average Travel Time to Work per person'] = sf_all_drop.loc[:, 'Average Travel Time to Work per person'].astype(float)
/var/folders/kz/dhtdp71n1q98lc7x715t7c0h0000gp/T/ipykernel_30522/2077931995.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sf_all_drop.loc[:, 'Average Travel Time to Work per person'] = sf_all_drop.loc[:, 'Average Travel Time to Work per person'].astype(floa

In [102]:
sf_all_drop

,Zip Code,Total Population,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Percentage of Rent,Year,Average Travel Time to Work per person
0,94401,34845,13606,12693,913,0.067103,2011,11.918353
1,94402,23408,9617,9356,261,0.027139,2011,11.343131
2,94403,39102,16190,15266,924,0.057072,2011,11.720244
3,94404,32986,13882,13157,725,0.052226,2011,12.094222
4,94063,30788,9340,8877,463,0.049572,2011,9.511823
...,...,...,...,...,...,...,...,...
562,94941,32009,14139,13290,849,0.060047,2019,14.139461
563,94965,11394,6364,5950,414,0.065053,2019,15.313323
564,94014,49170,14478,14044,434,0.029977,2019,17.490746
565,94025,43392,16365,15106,1259,0.076932,2019,11.890325


# Perform Linear Interpolation on SF

In [134]:
import pandas as pd
import numpy as np

def create_dataframe():
    df = (pd.DataFrame(index=pd.date_range('2010-01-01', '2019-12-31', freq='3MS'))
          .reset_index()
          .rename(columns={'index': 'y_dt'})
          .assign(key=1)
         )
    return df

def merge_dataframes(df, df_to_merge):
    df_census_tract = df_to_merge.loc[:, ['Zip Code']].drop_duplicates().assign(key=1)
    df = pd.merge(df, df_census_tract, how='outer', on='key').drop(['key'], axis=1)

    assert df_to_merge.shape[0] == df_to_merge.loc[:, ['Zip Code', 'Year']].drop_duplicates().shape[0]

    df_to_merge.loc[:, 'y_dt'] = pd.to_datetime(df_to_merge['Year'], format='%Y')
    df_to_merge = df_to_merge.sort_values('y_dt')

    merged_df = pd.merge(df, df_to_merge, how='left', on=['y_dt', 'Zip Code'])
    merged_df = merged_df.sort_values(['Zip Code', 'y_dt'])
    return merged_df

def interpolate_data(merged_df):
    new_df = merged_df.copy()
    new_df.loc[:, 'new_col'] = np.nan

    for census_tract in merged_df.loc[:, 'Zip Code'].unique():

        merged_df = merged_df.sort_values(['Zip Code', 'y_dt'])

        for col in ['Total Population', 'Total Housing','Owner-occupied housing units','Renter-occupied housing units',
                   'Average Travel Time to Work per person']:

            temp_df = (merged_df.loc[(merged_df.loc[:, 'Zip Code'] == census_tract), [col]]
                        .interpolate(method="spline", order=1, limit_direction="both", downcast="infer")
                        .interpolate(method='bfill')
                        .reset_index(drop=True))

            new_df.loc[(new_df.loc[:, 'Zip Code'] == census_tract), f'Quarterly {col}'] = temp_df.values
    return new_df


In [135]:
#call the functions

df = create_dataframe()
merged_df = merge_dataframes(df, sf_all_drop)
new_df = interpolate_data(merged_df)

/var/folders/kz/dhtdp71n1q98lc7x715t7c0h0000gp/T/ipykernel_30522/886831574.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_merge.loc[:, 'y_dt'] = pd.to_datetime(df_to_merge['Year'], format='%Y')
/var/folders/kz/dhtdp71n1q98lc7x715t7c0h0000gp/T/ipykernel_30522/886831574.py:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_to_merge.loc[:, 'y_dt'] = pd.to_datetime(df_to_merge['Year'], format='%Y')
/opt/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_fitpack2.py:298: UserWarning: 
The maximal number of it

In [136]:
new_df

,y_dt,Zip Code,Total Population,Total Housing,Owner-occupied housing units,Renter-occupied housing units,Percentage of Rent,Year,Average Travel Time to Work per person,new_col,Quarterly Total Population,Quarterly Total Housing,Quarterly Owner-occupied housing units,Quarterly Renter-occupied housing units,Quarterly Average Travel Time to Work per person
35,2010-01-01,94002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25455.737463,10936.248035,10326.132878,610.685562,11.411945
91,2010-04-01,94002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25541.267250,10936.159163,10332.503626,604.118004,11.488292
147,2010-07-01,94002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25626.797036,10936.070290,10338.874375,597.550446,11.564639
203,2010-10-01,94002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25712.326823,10935.981418,10345.245123,590.982887,11.640986
259,2011-01-01,94002,25797.0,10936.0,10352.0,584.0,0.053402,2011.0,12.086289,NaN,25797.000000,10936.000000,10352.000000,584.000000,12.086289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2018-10-01,94965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11397.501555,6387.766070,5938.738134,449.080656,14.869174
2067,2019-01-01,94965,11394.0,6364.0,5950.0,414.0,0.065053,2019.0,15.313323,NaN,11394.000000,6364.000000,5950.000000,414.000000,15.313323
2123,2019-04-01,94965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11391.286150,6340.388960,5961.375212,379.027021,14.920759
2179,2019-07-01,94965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11388.178447,6316.700404,5972.693750,344.000203,14.946552


In [143]:
cleaned_df = new_df.loc[:, ['y_dt','Zip Code','Quarterly Total Population','Quarterly Total Housing',
                           'Quarterly Owner-occupied housing units','Quarterly Renter-occupied housing units'
                           ,'Quarterly Average Travel Time to Work per person']]

In [151]:
#cleaned_df.iloc[:,2:6] = cleaned_df.iloc[:,2:6].astype(int)

,Quarterly Total Population,Quarterly Total Housing,Quarterly Owner-occupied housing units,Quarterly Renter-occupied housing units
35,25455.737463,10936.248035,10326.132878,610.685562
91,25541.267250,10936.159163,10332.503626,604.118004
147,25626.797036,10936.070290,10338.874375,597.550446
203,25712.326823,10935.981418,10345.245123,590.982887
259,25797.000000,10936.000000,10352.000000,584.000000
...,...,...,...,...
2011,11397.501555,6387.766070,5938.738134,449.080656
2067,11394.000000,6364.000000,5950.000000,414.000000
2123,11391.286150,6340.388960,5961.375212,379.027021
2179,11388.178447,6316.700404,5972.693750,344.000203


In [141]:
def clean_intepolate_df(new_df):
    new_df = new_df.drop(columns=['new_col','Year'])
    new_df.iloc[:, 2:6] = new_df.iloc[:, 2:6].astype(int)
    new_df.iloc[:, 8:12] = new_df.iloc[:, 8:12].astype(int)
    return new_df

In [152]:
cleaned_df.to_csv('May_Q_SF_ACS_2011-2019.csv',index=False)

# Concat four cities ACS data

In [ ]:

def final_acs():
    cities = ['Q_DALLAS_ACS_2010-2019.csv', 'Q_SF_ACS_2010-2019.csv', 'Q_ATL_ACS_2010-2019.csv', 'Q_BOS_ACS_2010-2019.csv']
    dfs = [pd.read_csv(city, parse_dates=['y_dt']).drop('Unnamed: 0', axis=1) for city in cities]
    final_acs = pd.concat(dfs).reset_index(drop=True)
    return final_acs